In [12]:
import spacy
from spacytextblob.spacytextblob import SpacyTextBlob
import pandas as pd

nlp = spacy.load('en_core_web_md')
nlp.add_pipe('spacytextblob')

df20 = pd.read_csv('../data/processed/df_2020_ohe.csv', usecols=['row hash','priorities and concerns','important learning','comments'])
df21 = pd.read_csv('../data/processed/df_2021_ohe.csv', usecols=["Can you please tell us your organisations current key priorities and or concerns",
                                                                 'Can you tell us a short story about your experiences',
                                                                 'Is there anything else you would like to share with us or comment on'])

In [40]:
def sentiment_extraction_from_text(df, lst, year):
    tmp = pd.DataFrame()
    for item in lst:
        text = []
        sentiment = []
        subjectivity = []
        sentences = df[item].dropna().to_list()
        for sentence in sentences:
            text.append(sentence)
            doc = nlp(sentence)
            sentiment.append(doc._.blob.polarity)
            subjectivity.append(doc._.blob.subjectivity)
        sentiment_group = ["Positive" if x >=0.4 else 'Neutral' if x>=-0.4 and x<0.4 else "Negative" for x in sentiment]
        subjectivity_group = ["More Subjective" if x >=0.5 else 'More Objective' for x in subjectivity]
        items = [item for x in text]
        temp = pd.DataFrame({'Field':items,
                    'Text': text,
                    'sentiment': sentiment,
                    'sentiment_group': sentiment_group,
                    'subjectivity': subjectivity,
                    'subjectivity_group': subjectivity_group})
        tmp = pd.concat([tmp, temp])
    tmp.to_csv(f'../data/processed/sentiment/{year}_sentiment_subjectivity.csv')
        

In [41]:
sentiment_extraction_from_text(df20, ['priorities and concerns','important learning','comments'], 2020)

In [42]:
sentiment_extraction_from_text(df21, ["Can you please tell us your organisations current key priorities and or concerns",
                                      'Can you tell us a short story about your experiences',
                                      'Is there anything else you would like to share with us or comment on'], 2021)